In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords


In [2]:
#loading the data

data=pd.read_csv('data/imdb_labelled.txt' , delimiter='\t', names=['review', 'sentiment'])

In [3]:
data.head()


,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     748 non-null    object
 1   sentiment  748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [5]:
data['sentiment'].value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [42]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words=[]
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("movie is not good")

'movie is not good'

In [75]:
train_data = data[:int(0.5*len(data))]
test_and_dev_data = data[int(0.5*len(data)):]


dev_data=test_and_dev_data[:int(0.5 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.5 * len(test_and_dev_data)):]

print(train_data.info())

print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     374 non-null    object
 1   sentiment  374 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.0+ KB
None



In [76]:
print(test_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 561 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     187 non-null    object
 1   sentiment  187 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ KB
None


In [77]:
print(dev_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 374 to 560
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     187 non-null    object
 1   sentiment  187 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ KB
None


In [78]:
train_data['sentiment'].value_counts()

0    221
1    153
Name: sentiment, dtype: int64

In [79]:
test_data['sentiment'].value_counts()

1    129
0     58
Name: sentiment, dtype: int64

In [80]:
dev_data['sentiment'].value_counts()

1    104
0     83
Name: sentiment, dtype: int64

In [81]:
X_train = train_data['review']
y_train = train_data['sentiment']
# print(X_train)
# print(y_train)

X_dev = dev_data['review']
y_dev = dev_data['sentiment']
# print(X_dev)
# print(y_dev)

X_test = test_data['review']
y_test = test_data['sentiment']
# print(X_test)
# print(y_test)

In [82]:

def cal_prior_probs(y_train):
    prior_probs = []
    count_0 = sum(y_train == 0)
    prior_probs.append(count_0 / y_train.size)  
    count_1 = sum(y_train == 1)
    prior_probs.append(count_1 / y_train.size)    
    return prior_probs
prior_probs=cal_prior_probs(y_train)
print("Negative review class' prior probability--",prior_probs[0] )
print("Positive review class' prior probability--",prior_probs[1] )

Negative review class' prior probability-- 0.5909090909090909
Positive review class' prior probability-- 0.4090909090909091


In [83]:
def build_word_dict(data):
    word_freq = {}
    all_words=[]
    omit=[]
    for sentence in data:
            words=[]
            sentence = text_filtering(sentence)
            words=sentence.split(' ')
            for word in set(words) :
                word = word.lower()
                if word not in word_freq.keys():
                    word_freq[word] = 1
                    all_words.append(word)
                else:
                    word_freq[word] += 1
#     for word in word_freq:
#         if word_freq[word] <5:
#             omit.append(word)

#     for word in omit:
#         word_freq.pop(word,None)
        
            

    return word_freq, all_words

    
       
allwords_freq,allwords_train=build_word_dict(X_train)
print("Total words--",len(allwords_train))


train_pos_reviews=train_data.loc[train_data.sentiment==1].review
pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
print("Positive words--",len(pos_words))


train_neg_reviews=train_data.loc[train_data.sentiment==0].review
neg_words_freq,neg_words=build_word_dict(train_neg_reviews)
print("Negative words--",len(neg_words))

Total words-- 2055
Positive words-- 907
Negative words-- 1563


In [84]:
allwords_freq.pop('', None)

for word in allwords_freq:
    print(word,"--",allwords_freq[word],"    probability=",allwords_freq[word]/len(X_train))

a -- 134     probability= 0.3582887700534759
very -- 26     probability= 0.06951871657754011
about -- 20     probability= 0.053475935828877004
distressed -- 1     probability= 0.00267379679144385
slowmoving -- 1     probability= 0.00267379679144385
drifting -- 1     probability= 0.00267379679144385
young -- 3     probability= 0.008021390374331552
movie -- 76     probability= 0.20320855614973263
man -- 8     probability= 0.0213903743315508
aimless -- 1     probability= 0.00267379679144385
lost -- 2     probability= 0.0053475935828877
sure -- 3     probability= 0.008021390374331552
or -- 20     probability= 0.053475935828877004
not -- 29     probability= 0.07754010695187166
characters -- 15     probability= 0.040106951871657755
nearly -- 1     probability= 0.00267379679144385
walked -- 1     probability= 0.00267379679144385
of -- 111     probability= 0.2967914438502674
who -- 18     probability= 0.0481283422459893
out -- 23     probability= 0.06149732620320856
flat -- 2     probability= 

fact -- 2     probability= 0.0053475935828877
tremendously -- 2     probability= 0.0053475935828877
wifetobe -- 1     probability= 0.00267379679144385
affleck -- 1     probability= 0.00267379679144385
ben -- 1     probability= 0.00267379679144385
between -- 5     probability= 0.013368983957219251
couldnt -- 3     probability= 0.008021390374331552
sandra -- 1     probability= 0.00267379679144385
bullock -- 1     probability= 0.00267379679144385
leaving -- 1     probability= 0.00267379679144385
understand -- 4     probability= 0.0106951871657754
chemistry -- 2     probability= 0.0053475935828877
knocked -- 1     probability= 0.00267379679144385
why -- 4     probability= 0.0106951871657754
supposedly -- 1     probability= 0.00267379679144385
moving -- 1     probability= 0.00267379679144385
moments -- 2     probability= 0.0053475935828877
slow -- 1     probability= 0.00267379679144385
several -- 2     probability= 0.0053475935828877
need -- 1     probability= 0.00267379679144385
excruciati

exemplars -- 1     probability= 0.00267379679144385
explorations -- 1     probability= 0.00267379679144385
powerful -- 1     probability= 0.00267379679144385
stories -- 1     probability= 0.00267379679144385
learn -- 1     probability= 0.00267379679144385
japanese -- 1     probability= 0.00267379679144385
politics -- 1     probability= 0.00267379679144385
rendering -- 1     probability= 0.00267379679144385
americas -- 1     probability= 0.00267379679144385
imperial -- 1     probability= 0.00267379679144385
values -- 2     probability= 0.0053475935828877
underappreciated -- 1     probability= 0.00267379679144385
makes -- 4     probability= 0.0106951871657754
production -- 1     probability= 0.00267379679144385
faultless -- 1     probability= 0.00267379679144385
teddy -- 1     probability= 0.00267379679144385
keith -- 1     probability= 0.00267379679144385
vivid -- 1     probability= 0.00267379679144385
composition -- 1     probability= 0.00267379679144385
brian -- 2     probability= 0.0

In [85]:
neg_words_freq.pop('', None)
neg_condi_prob={}
for word in neg_words_freq:
    print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
    neg_condi_prob[word]=neg_words_freq[word]/len(train_neg_reviews)

a -- 72       Conditional probability of a  given sentiment is neg= 0.3257918552036199
very -- 17       Conditional probability of very  given sentiment is neg= 0.07692307692307693
about -- 14       Conditional probability of about  given sentiment is neg= 0.06334841628959276
distressed -- 1       Conditional probability of distressed  given sentiment is neg= 0.004524886877828055
slowmoving -- 1       Conditional probability of slowmoving  given sentiment is neg= 0.004524886877828055
drifting -- 1       Conditional probability of drifting  given sentiment is neg= 0.004524886877828055
young -- 2       Conditional probability of young  given sentiment is neg= 0.00904977375565611
movie -- 43       Conditional probability of movie  given sentiment is neg= 0.19457013574660634
man -- 4       Conditional probability of man  given sentiment is neg= 0.01809954751131222
aimless -- 1       Conditional probability of aimless  given sentiment is neg= 0.004524886877828055
lost -- 1       Conditional

made -- 9       Conditional probability of made  given sentiment is neg= 0.04072398190045249
energy -- 3       Conditional probability of energy  given sentiment is neg= 0.013574660633484163
right -- 2       Conditional probability of right  given sentiment is neg= 0.00904977375565611
character -- 6       Conditional probability of character  given sentiment is neg= 0.027149321266968326
generic -- 1       Conditional probability of generic  given sentiment is neg= 0.004524886877828055
hanks -- 1       Conditional probability of hanks  given sentiment is neg= 0.004524886877828055
arent -- 2       Conditional probability of arent  given sentiment is neg= 0.00904977375565611
story -- 4       Conditional probability of story  given sentiment is neg= 0.01809954751131222
screen -- 2       Conditional probability of screen  given sentiment is neg= 0.00904977375565611
just -- 16       Conditional probability of just  given sentiment is neg= 0.07239819004524888
itself -- 2       Conditional pro

boring -- 4       Conditional probability of boring  given sentiment is neg= 0.01809954751131222
occupied -- 2       Conditional probability of occupied  given sentiment is neg= 0.00904977375565611
watching -- 6       Conditional probability of watching  given sentiment is neg= 0.027149321266968326
happened -- 2       Conditional probability of happened  given sentiment is neg= 0.00904977375565611
sometimes -- 3       Conditional probability of sometimes  given sentiment is neg= 0.013574660633484163
myself -- 2       Conditional probability of myself  given sentiment is neg= 0.00904977375565611
instead -- 1       Conditional probability of instead  given sentiment is neg= 0.004524886877828055
columbo -- 1       Conditional probability of columbo  given sentiment is neg= 0.004524886877828055
look -- 5       Conditional probability of look  given sentiment is neg= 0.02262443438914027
seems -- 1       Conditional probability of seems  given sentiment is neg= 0.004524886877828055
misplace 

yeah -- 2       Conditional probability of yeah  given sentiment is neg= 0.00904977375565611
sucked -- 3       Conditional probability of sucked  given sentiment is neg= 0.013574660633484163
storyline -- 4       Conditional probability of storyline  given sentiment is neg= 0.01809954751131222
pillow -- 1       Conditional probability of pillow  given sentiment is neg= 0.004524886877828055
keep -- 1       Conditional probability of keep  given sentiment is neg= 0.004524886877828055
through -- 4       Conditional probability of through  given sentiment is neg= 0.01809954751131222
girlfriendboyfriend -- 1       Conditional probability of girlfriendboyfriend  given sentiment is neg= 0.004524886877828055
gone -- 1       Conditional probability of gone  given sentiment is neg= 0.004524886877828055
disliked -- 2       Conditional probability of disliked  given sentiment is neg= 0.00904977375565611
five -- 1       Conditional probability of five  given sentiment is neg= 0.004524886877828055
ma

started -- 4       Conditional probability of started  given sentiment is neg= 0.01809954751131222
ironically -- 1       Conditional probability of ironically  given sentiment is neg= 0.004524886877828055
fails -- 1       Conditional probability of fails  given sentiment is neg= 0.004524886877828055
soul -- 1       Conditional probability of soul  given sentiment is neg= 0.004524886877828055
greatness -- 1       Conditional probability of greatness  given sentiment is neg= 0.004524886877828055
roths -- 1       Conditional probability of roths  given sentiment is neg= 0.004524886877828055
end -- 2       Conditional probability of end  given sentiment is neg= 0.00904977375565611
manages -- 1       Conditional probability of manages  given sentiment is neg= 0.004524886877828055
provided -- 1       Conditional probability of provided  given sentiment is neg= 0.004524886877828055
turn -- 2       Conditional probability of turn  given sentiment is neg= 0.00904977375565611
ranks -- 1       Co

created -- 1       Conditional probability of created  given sentiment is neg= 0.004524886877828055
opening -- 1       Conditional probability of opening  given sentiment is neg= 0.004524886877828055
cg -- 1       Conditional probability of cg  given sentiment is neg= 0.004524886877828055
slideshow -- 1       Conditional probability of slideshow  given sentiment is neg= 0.004524886877828055
gas -- 1       Conditional probability of gas  given sentiment is neg= 0.004524886877828055
tanks -- 1       Conditional probability of tanks  given sentiment is neg= 0.004524886877828055
explosion -- 1       Conditional probability of explosion  given sentiment is neg= 0.004524886877828055
avoided -- 1       Conditional probability of avoided  given sentiment is neg= 0.004524886877828055
virus -- 1       Conditional probability of virus  given sentiment is neg= 0.004524886877828055
sum -- 1       Conditional probability of sum  given sentiment is neg= 0.004524886877828055
breeders -- 1       Condit

In [86]:
neg_condi_prob

{'a': 0.3257918552036199,
 'very': 0.07692307692307693,
 'about': 0.06334841628959276,
 'distressed': 0.004524886877828055,
 'slowmoving': 0.004524886877828055,
 'drifting': 0.004524886877828055,
 'young': 0.00904977375565611,
 'movie': 0.19457013574660634,
 'man': 0.01809954751131222,
 'aimless': 0.004524886877828055,
 'lost': 0.004524886877828055,
 'sure': 0.013574660633484163,
 'or': 0.06787330316742081,
 'not': 0.1085972850678733,
 'characters': 0.04524886877828054,
 'nearly': 0.004524886877828055,
 'walked': 0.004524886877828055,
 'of': 0.29411764705882354,
 'who': 0.049773755656108594,
 'out': 0.049773755656108594,
 'flat': 0.00904977375565611,
 'half': 0.02262443438914027,
 'whom': 0.00904977375565611,
 'more': 0.03619909502262444,
 'was': 0.23076923076923078,
 'audience': 0.00904977375565611,
 'the': 0.6108597285067874,
 'with': 0.1085972850678733,
 'artiness': 0.004524886877828055,
 'angles': 0.004524886877828055,
 'disappointed': 0.01809954751131222,
 'as': 0.0950226244343891

In [87]:
pos_words_freq.pop('', None)
pos_condi_prob={}
for word in pos_words_freq:
    print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",pos_words_freq[word]/len(train_pos_reviews))
    pos_condi_prob[word]=pos_words_freq[word]/len(train_pos_reviews)

song -- 1       Conditional probability of song  given sentiment is neg= 0.006535947712418301
scene -- 1       Conditional probability of scene  given sentiment is neg= 0.006535947712418301
a -- 62       Conditional probability of a  given sentiment is neg= 0.40522875816993464
through -- 3       Conditional probability of through  given sentiment is neg= 0.0196078431372549
trying -- 1       Conditional probability of trying  given sentiment is neg= 0.006535947712418301
the -- 93       Conditional probability of the  given sentiment is neg= 0.6078431372549019
when -- 7       Conditional probability of when  given sentiment is neg= 0.0457516339869281
running -- 1       Conditional probability of running  given sentiment is neg= 0.006535947712418301
to -- 37       Conditional probability of to  given sentiment is neg= 0.24183006535947713
best -- 9       Conditional probability of best  given sentiment is neg= 0.058823529411764705
is -- 43       Conditional probability of is  given sentime

series -- 1       Conditional probability of series  given sentiment is neg= 0.006535947712418301
showthese -- 1       Conditional probability of showthese  given sentiment is neg= 0.006535947712418301
smack -- 1       Conditional probability of smack  given sentiment is neg= 0.006535947712418301
writers -- 1       Conditional probability of writers  given sentiment is neg= 0.006535947712418301
bonus -- 1       Conditional probability of bonus  given sentiment is neg= 0.006535947712418301
actresses -- 1       Conditional probability of actresses  given sentiment is neg= 0.006535947712418301
myself -- 3       Conditional probability of myself  given sentiment is neg= 0.0196078431372549
again -- 3       Conditional probability of again  given sentiment is neg= 0.0196078431372549
say -- 2       Conditional probability of say  given sentiment is neg= 0.013071895424836602
taped -- 1       Conditional probability of taped  given sentiment is neg= 0.006535947712418301
episodes -- 1       Cond

rendering -- 1       Conditional probability of rendering  given sentiment is neg= 0.006535947712418301
americas -- 1       Conditional probability of americas  given sentiment is neg= 0.006535947712418301
imperial -- 1       Conditional probability of imperial  given sentiment is neg= 0.006535947712418301
here -- 3       Conditional probability of here  given sentiment is neg= 0.0196078431372549
values -- 1       Conditional probability of values  given sentiment is neg= 0.006535947712418301
underappreciated -- 1       Conditional probability of underappreciated  given sentiment is neg= 0.006535947712418301
makes -- 3       Conditional probability of makes  given sentiment is neg= 0.0196078431372549
production -- 1       Conditional probability of production  given sentiment is neg= 0.006535947712418301
faultless -- 1       Conditional probability of faultless  given sentiment is neg= 0.006535947712418301
teddy -- 1       Conditional probability of teddy  given sentiment is neg= 0.006

In [88]:
pos_condi_prob

{'song': 0.006535947712418301,
 'scene': 0.006535947712418301,
 'a': 0.40522875816993464,
 'through': 0.0196078431372549,
 'trying': 0.006535947712418301,
 'the': 0.6078431372549019,
 'when': 0.0457516339869281,
 'running': 0.006535947712418301,
 'to': 0.24183006535947713,
 'best': 0.058823529411764705,
 'is': 0.28104575163398693,
 'was': 0.17647058823529413,
 'find': 0.0196078431372549,
 'head': 0.006535947712418301,
 'that': 0.13071895424836602,
 'keeps': 0.006535947712418301,
 'in': 0.20915032679738563,
 'movie': 0.21568627450980393,
 'gerardo': 0.006535947712418301,
 'his': 0.0392156862745098,
 'for': 0.10457516339869281,
 'it': 0.26143790849673204,
 'today': 0.013071895424836602,
 'and': 0.43137254901960786,
 'effort': 0.006535947712418301,
 'saw': 0.0196078431372549,
 'messages': 0.006535947712418301,
 'good': 0.08496732026143791,
 'thought': 0.013071895424836602,
 'kids': 0.013071895424836602,
 'teacher': 0.006535947712418301,
 'casting': 0.0196078431372549,
 'buffet': 0.0065359

In [89]:
def score(real, prediction):
    count = 0
    for i in range(len(real)):
        if real[i] == prediction[i]:
            count+=1
    accuracy=count/len(real)*100
    return accuracy

In [90]:
def pred(data,ans):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=0
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=0
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train) )   
print("Accuracy on development data--",pred(X_dev,y_dev))
print("Accuracy on test data--",pred(X_test,y_test))

Accuracy on training data-- 92.24598930481284
Accuracy on development data-- 49.73262032085562
Accuracy on test data-- 33.155080213903744
